In [1]:
# main script for reading the matrix csv file and learning off of the given matrix
# directly from https://realpython.com/python-csv/#reading-csv-files-with-csv
import csv
import numpy as np

In [3]:
with open('matrix_data.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    data = list(csv_reader)
    data = data[0]
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')

Processed 0 lines.


In [4]:
epsilon = float(data[0])
discount_factor = float(data[1])
learning_rate = float(data[2])
width = int(float(data[3]))
height = int(float(data[4]))

In [5]:
# printing hyper parameters
print("Epsilon:", epsilon)
print("Discount factor:", discount_factor)
print("Learning rate:", learning_rate)
print("Rows:", height)
print("Columns:", width)

Epsilon: 1.0
Discount factor: 0.99
Learning rate: 0.01
Rows: 11
Columns: 11


In [6]:
values = np.zeros((height, width))

index = 5 # start at index 5 to skip parameters
for row in range(height):
    for col in range(width):
        values[row][col] = data[index]
        index += 1

print(values)

[[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
 [-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
 [-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
 [-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]]


In [23]:
actions = ['up', 'right', 'down', 'left']
num_actions = 4

In [37]:
# Start of Q-Learning implementation
# define helper functions

# checks if the given row, col state is terminal
def is_terminal(values, row, col):
    # -1 defined as the value of squares that are non-terminal
    nonterminal_value = -1
    if values[row][col] == nonterminal_value:
        return False
    else:
        return True

# gets a random valid start state given max rows and cols
def get_rand_start(values, max_rows, max_cols):
    start_row = np.random.randint(max_rows)
    start_col = np.random.randint(max_cols)
    while is_terminal(values, start_row, start_col):
        start_row = np.random.randint(max_rows)
        start_col = np.random.randint(max_cols)
    return start_row, start_col

# get the next action following an epsilon greedy choice
def get_next_action(q_table, epsilon, row, col):
    # return greedy action
    if np.random.random() < epsilon:
        action_num = np.argmax(q_table[row][col])
    # return random action
    else:
        action_num = np.random.randint(num_actions)
    return action_num

# get the next location. if invalid, don't move
def get_next_location(old_row, old_col, max_rows, max_cols, action_num):
    # check U, D, L, R
    new_row = old_row
    new_col = old_col
    action_name = actions[action_num]
    if action_name == 'up' and old_row > 0:
        new_row = old_row - 1
    elif action_name == 'right' and old_col < max_cols - 1:
        new_col = old_col + 1
    elif action_name == 'down' and old_row < max_rows - 1:
        new_row = old_row + 1
    elif action_name == 'left' and old_col > 0:
        new_col = old_col - 1
    # else invalid move -> dont move
    return new_row, new_col

# get the best path following the Q-table
def get_best_path(q_table, values, start_row, start_col):
    path = []
    curr_row, curr_col = start_row, start_col
    # keep appending while not terminal
    while not is_terminal(values, curr_row, curr_col):
        path.append([curr_row, curr_col])
        action_num = get_next_action(q_table, 1, curr_row, curr_col)
        curr_row, curr_col = get_next_location(curr_row, curr_col, height, width, action_num)
    # append the terminal state as well
    path.append([curr_row, curr_col])
    return path

In [38]:
# define Q-table
q_table = np.zeros((height, width, num_actions))